# Atributos estáticos

Um atributo estático é um atributo que pertence à classe, e não ao objeto instanciado daquela classe. Comumente chamamos tal atributo de `atributo de classe` ou `variável de classe`

&nbsp;

Sua declaração é feita fora do construtor da classe - diretamente na classe - e é acessível independente da criação da instância do objeto.

```python
class Teste:
    atributo_estatico = nome
```

Para acessá-la podemos utilizar a sintaxe abaixo, sem nem mesmo termos objetos instanciados na classe:

&nbsp;

```python
nome_da_classe.nome_do_atributo
```

&nbsp;



&nbsp;

Considerando nossa modelagem de conta corrente/titulares, podemos utilizar o atributo estático para controlar quais CPFs já foram utilizados no cadastro  de titulares (visando assim não permitir CPFs duplicados no sistema).

In [ ]:
from datetime import date
from typing import List, Dict

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str, cpf: str, dt_nasc: date):
        self._nome: str = nome
        self._cpf: str = cpf
        Titular.cpfs_utilizados.append(cpf)
        self._dt_nasc: date = dt_nasc

    @property
    def nome_titular(self) -> str:
        return self._nome

    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

class Conta:
  def __init__(self, titular: Titular, agencia: str, conta: str):
        self._titular: Titular = titular
        self._agencia: str = agencia
        self._conta: str = conta
        self._saldo: float = 0.0
        self._extrato: List[Dict[str, str]] = []

  @property
  def titular(self) -> Titular:
      return self._titular

  @property
  def agencia(self) -> str:
      return self._agencia

  @property
  def conta(self) -> str:
      return self._conta

  @property
  def saldo(self) -> float:
      return self._saldo

  def _adicionar_extrato(self, tipo: str, valor: float):
      valor_formatado = '{:.2f}'.format(valor)
      self._extrato.append({'key': tipo.upper(), 'value': valor_formatado})

  def _msg_resposta(self, sucesso: bool, nome_operacao: str) -> None:
      if sucesso:
          print(f'Operação realizada com sucesso. Operação: {nome_operacao}')
      else:
          print(f'Falha ao realizar operação. Operação: {nome_operacao}')

  def _saidas(self, valor: float, nome_operacao: str) -> bool:
      if self._saldo >= valor:
          self._saldo -= valor
          self._adicionar_extrato(tipo='s', valor=valor)
          self._msg_resposta(sucesso=True, nome_operacao=nome_operacao)
          return True
      else:
          self._msg_resposta(sucesso=False, nome_operacao=nome_operacao)
          return False

  def deposito(self, valor: float) -> None:
      nome_operacao = 'Deposito'
      if valor > 0.0:
          self._saldo += valor
          self._adicionar_extrato(tipo='e', valor=valor)
          self._msg_resposta(sucesso=True, nome_operacao=nome_operacao)
      else:
          self._msg_resposta(sucesso=False, nome_operacao=nome_operacao)

  def saque(self, valor: float) -> None:
      self._saidas(valor=valor, nome_operacao='Saque')

  def extrato(self):
      print(f'Agencia: {self._agencia}')
      print(f'Conta: {self._conta}')
      print(f'Titular: {self.titular._nome}')
      print(f'CPF do titular: {self._titular.cpf}')
      print('Saldo: R$', '{:.2f}'.format(self._saldo), sep=' ')
      for mov in self._extrato:
          print(f'\t{mov["key"]}: R$ {mov["value"]}')

class ContaCorrente(Conta):
  def __init__(self, titular: Titular, agencia: str, conta: str):
      super().__init__(titular, agencia, conta)

  def pagamento(self, valor: float) -> None:
      self._saidas(valor=valor, nome_operacao='Pagamento')

  def transferencia(self, valor: float, conta_destino: Conta) -> None:
      nome_operacao = 'Transferencia'
      if self._saldo >= valor:
          self._saldo -= valor
          self._adicionar_extrato(tipo='s', valor=valor)
          conta_destino.deposito(valor)
          self._msg_resposta(sucesso=True, nome_operacao=nome_operacao)
      else:
          self._msg_resposta(sucesso=False, nome_operacao=nome_operacao)

class ContaPoupanca(Conta):
  def __init__(self, titular: Titular, agencia: str, conta: str):
        super().__init__(titular, agencia, conta)

  def saque(self, valor: float) -> None:
      print('Operação bloqueada pelo BACEN.')

In [ ]:
dt_nasc = date(year=1991, month=8, day=6)
t1 = Titular('Pedro', '12345678900', dt_nasc)

Titular.cpfs_utilizados

Porém, como controlar de fato e validar se o CPF já foi inserido?

Podemos:

- Validar os CPFs no nosso programa principal

In [ ]:
cpf = 12345678900
if cpf in Titular.cpfs_utilizados:
  print(f'CPF {cpf} já cadastrado no sistema.')
else:
  dt_nasc = date(year=1991, month=8, day=6)
  t1 = Titular('Pedro', cpf, dt_nasc)

Titular.cpfs_utilizados

- Validar o CPF no construtor da classe Titular:

In [ ]:
from typing import List
from datetime import date

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str, cpf: str, dt_nasc: date):
        if cpf in Titular.cpfs_utilizados:
          print(f'CPF {cpf} já cadastrado no sistema.')
          return
        self._nome: str = nome
        self._cpf: str = cpf
        Titular.cpfs_utilizados.append(cpf)
        self._dt_nasc: date = dt_nasc

    @property
    def nome_titular(self) -> str:
        return self._nome

    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

In [ ]:
cpf = 12345678900
dt_nasc = date(year=1991, month=8, day=6)
t1 = Titular('Pedro', cpf, dt_nasc)

Titular.cpfs_utilizados

Em ambos os casos o atributo estático armazena corretamente os CPFs já cadastrados e não permite valores repetidos. Porém, conseguem observar algum problema com o uso do atributo estático?

&nbsp;

Exatamente, ele é invocado usando

```python
nome_da_classe.nome_do_atributo
```

Agora imaginem que seja necessário refatorar o nome de uma ou mais classes, e ela contém tais atributos... pois é, vamos precisar sair caçando no código quem mais usa a classe X para refatorar o nome.

&nbsp;

O python fornece outra forma de lidar com a necessidade de invocar um método sem que tenhamos um objeto, que são os métodos de classe.



# Métodos de classe

Da mesma forma que as instâncias de objetos possuem os métodos convencionais (os métodos de objetos, com `self` como primeiro parâmetro), podemos criar métodos que recebem a própria classe como parâmetro (chamada normalmente de `cls`).

Dessa forma, podemos alterar o estado da classe (por exemplo, atualizar um atributo estático) sem precisar utilizar o nome da classe diretamente.

Tais métodos são criados conforme a seguinte sintaxe:

```python
@classmethod
def metodo_de_classe(cls, parametros):
```

> Importante: nos métodos de classe passamos a referência à classe usando normalmente o `cls`

&nbsp;

Utilizamos um método de classe conforme a sintaxe abaixo:

&nbsp;

```python
instancia_do_objeto.nome_do_metodo_da_classe()
```

&nbsp;


In [ ]:
from typing import List
from datetime import date

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str, cpf: str, dt_nasc: date):
        if self.cpf_ja_utilizado(cpf):
          return
        else:
          self.adicionar_cpf(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc

    @property
    def nome_titular(self) -> str:
        return self._nome

    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

    @classmethod
    def cpf_ja_utilizado(cls, cpf: str):
      if cpf in cls.cpfs_utilizados:
          print(f'CPF {cpf} já cadastrado no sistema.')
          return True
      return False

    @classmethod
    def adicionar_cpf(cls, cpf: str):
      cls.cpfs_utilizados.append(cpf)

    @classmethod
    def listar_cpfs(cls):
      print(cls.cpfs_utilizados)

In [ ]:
cpf = '12345678900'
dt_nasc = date(year=1991, month=8, day=6)
t1 = Titular('Pedro', cpf, dt_nasc)

t1.listar_cpfs()

print()

cpf = '98765432100'
dt_nasc = date(year=1995, month=6, day=20)
t2 = Titular('Laura', cpf, dt_nasc)

t2.listar_cpfs()

Um outro uso dos métodos de classe é relacionado ao Padrão de Projeto Factory

Dado que no Python não temos a sobrecarga de construtores, como outras linguagens de programação possuem, podemos implementar um comportamento semelhante com os métodos de classe.

Observem o exemplo abaixo, onde vamos modificar um pouco o construtor da classe Titular para permitir instanciar o titular com parâmetros diferentes.

In [ ]:
from typing import List
from datetime import date

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str = None, cpf: str = None, dt_nasc: date = None):
        if self.cpf_ja_utilizado(cpf):
          return
        else:
          self.adicionar_cpf(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc

    @property
    def nome_titular(self) -> str:
        return self._nome

    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

    @classmethod
    def cpf_ja_utilizado(cls, cpf: str):
      if cpf and cpf in cls.cpfs_utilizados:
          print(f'CPF {cpf} já cadastrado no sistema.')
          return True
      return False

    @classmethod
    def adicionar_cpf(cls, cpf: str):
      cls.cpfs_utilizados.append(cpf)

    @classmethod
    def listar_cpfs(cls):
      print(cls.cpfs_utilizados)

    @classmethod
    def c1(cls, nome: str):
      return cls(nome=nome, cpf=None, dt_nasc=None)

    @classmethod
    def c2(cls, cpf: str):
      return cls(nome=None, cpf=cpf, dt_nasc=None)

    @classmethod
    def c3(cls, dt_nasc: date):
      return cls(nome=None, cpf=None, dt_nasc=dt_nasc)

    @classmethod
    def c4(cls, nome: str, cpf: str):
      return cls(nome=nome, cpf=cpf, dt_nasc=None)

    @classmethod
    def c5(cls, nome: str, dt_nasc: date):
      return cls(nome=nome, cpf=None, dt_nasc=dt_nasc)

    @classmethod
    def c6(cls, cpf: str, dt_nasc: date):
      return cls(nome=None, cpf=cpf, dt_nasc=dt_nasc)

In [ ]:
t0 = Titular(nome='Pedro', cpf='12345678900', dt_nasc=date(year=1991, month=8, day=6))
t0.listar_cpfs()
print()

t1 = Titular.c1(nome='Luana')
print(t1.nome_titular)
print(t1.cpf)
print(t1.data_nascimento)
print()

t2 = Titular.c2(cpf='85214796357')
print(t2.nome_titular)
print(t2.cpf)
print(t2.data_nascimento)
print()

t3 = Titular.c3(dt_nasc=date(year=1991, month=8, day=6))
print(t3.nome_titular)
print(t3.cpf)
print(t3.data_nascimento)
print()

t4 = Titular.c4(nome='Joana', cpf='96374185279')
print(t4.nome_titular)
print(t4.cpf)
print(t4.data_nascimento)
print()

t5 = Titular.c5(nome='Joana', dt_nasc=date(year=1991, month=8, day=6))
print(t5.nome_titular)
print(t5.cpf)
print(t5.data_nascimento)
print()

t6 = Titular.c6(cpf='14896352478', dt_nasc=date(year=1991, month=8, day=6))
print(t6.nome_titular)
print(t6.cpf)
print(t6.data_nascimento)
print()

-----------------------

# Métodos estáticos

Podemos também criar a validação do CPF como um método estático.

Um método estático não deveria afetar o estado de um objeto, tampouco da classe. Ele poderia tranquilamente ser removido de dentro da classe e implementado como uma função "avulsa" e seu funcionamento não seria prejudicado.

Neste caso, por que eles existem? A ideia está relacionada à noção de namespace: implementamos um método estático quando temos uma função que está relacionada ao assunto de nossa classe, e por conta disso gostaríamos de "agrupá-la" com a classe e ter seu nome vinculado ao nome da própria classe.


Sua declaração é feita em qualquer local da classe, tal qual os métodos comuns da classe, mas utilizando a seguinte sintaxe:

```python
@staticmethod
def metodo_estatico(parametros):
```

> Importante: nos métodos estáticos, não passamos a referência ao objeto `self` e nem a referência da classe `cls`

&nbsp;

Da mesma forma que o atributo estático é acessível independente da criação da instância do objeto, o método estático também é. Utilizamos um método estático conforme a sintaxe abaixo:

&nbsp;

```python
nome_da_classe.nome_do_metodo(parametros)
```

&nbsp;


In [ ]:
from typing import List
from datetime import date

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str = None, cpf: str = None, dt_nasc: date = None):
        if self.cpf_ja_utilizado(cpf):
          return
        else:
          self.adicionar_cpf(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc

    @property
    def nome_titular(self) -> str:
        return self._nome

    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

    @staticmethod
    def validar_cpf(cpf):
      if cpf and len(cpf) == 11:
        return True
      else:
        return False

    @classmethod
    def cpf_ja_utilizado(cls, cpf: str):
      if Titular.validar_cpf(cpf) and cpf in cls.cpfs_utilizados:
          print(f'CPF {cpf} já cadastrado no sistema.')
          return True
      return False

    @classmethod
    def adicionar_cpf(cls, cpf: str):
      cls.cpfs_utilizados.append(cpf)

    @classmethod
    def listar_cpfs(cls):
      print(cls.cpfs_utilizados)

    @classmethod
    def c1(cls, nome: str):
      return cls(nome=nome, cpf=None, dt_nasc=None)

    @classmethod
    def c2(cls, cpf: str):
      return cls(nome=None, cpf=cpf, dt_nasc=None)

    @classmethod
    def c3(cls, dt_nasc: date):
      return cls(nome=None, cpf=None, dt_nasc=dt_nasc)

    @classmethod
    def c4(cls, nome: str, cpf: str):
      return cls(nome=nome, cpf=cpf, dt_nasc=None)

    @classmethod
    def c5(cls, nome: str, dt_nasc: date):
      return cls(nome=nome, cpf=None, dt_nasc=dt_nasc)

    @classmethod
    def c6(cls, cpf: str, dt_nasc: date):
      return cls(nome=None, cpf=cpf, dt_nasc=dt_nasc)

In [ ]:
t0 = Titular(nome='Pedro', cpf='12345678900', dt_nasc=date(year=1991, month=8, day=6))
t0.listar_cpfs()
print()

t1 = Titular.c1(nome='Luana')
print(t1.nome_titular)
print(t1.cpf)
print(t1.data_nascimento)
print()

t2 = Titular.c2(cpf='85214796357')
print(t2.nome_titular)
print(t2.cpf)
print(t2.data_nascimento)
print()

t3 = Titular.c3(dt_nasc=date(year=1991, month=8, day=6))
print(t3.nome_titular)
print(t3.cpf)
print(t3.data_nascimento)
print()

t4 = Titular.c4(nome='Joana', cpf='96374185279')
print(t4.nome_titular)
print(t4.cpf)
print(t4.data_nascimento)
print()

t5 = Titular.c5(nome='Joana', dt_nasc=date(year=1991, month=8, day=6))
print(t5.nome_titular)
print(t5.cpf)
print(t5.data_nascimento)
print()

t6 = Titular.c6(cpf='14896352478', dt_nasc=date(year=1991, month=8, day=6))
print(t6.nome_titular)
print(t6.cpf)
print(t6.data_nascimento)
print()

print(Titular.validar_cpf('123548647'))

------------------------

# Considerações

Um ponto que já foi amplamente discutido ao longo deste material é que o Python frequentemente permite muita coisa, mas isso nem sempre significa que você deveria fazer: por exemplo, alterar diretamente o valor de um atributo protected.

&nbsp;


Alguns conceitos similares se aplicam aqui. Por exemplo:

- É possível um `método estático` alterar o estado de uma classe (acessando diretamente através do nome da classe).

- É possível criar métodos sem `self`, sem `cls` e sem os decorators sinalizando que se são métodos de classe ou estáticos.

&nbsp;

Aqui vai valer um pouco a questão de bom senso: apesar de tudo isso ser possível, você não deve fazer isso.

- Ao utilizarmos o decorator `@classmethod` nós estamos sinalizando para outros desenvolvedores que aquele método irá lidar com conteúdo interno da classe. Além disso, o `@classmethod` irá sempre receber no primeiro parâmetro a classe. Isso evita a repetição do nome da classe (que já discutimos o motivo de ser indesejável) e evita que caso o método seja chamado a partir de um objeto ele receba o objeto como parâmetro ao invés da classe (ou seja, que o cls seja tratado como self).

- Ao utilizarmos o decorator `@staticmethod`, além de sinalizar para outros desenvolvedores que o método não utilizará o conteúdo interno da classe, nós também estamos sinalizando para o Python que ele não deve utilizar "quem chamou" o método como parâmetro.
